In [38]:
import tensorflow as tf
import pandas as pd
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D 
from keras.layers import Add
from keras.layers.advanced_activations import PReLU
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot 


from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model


import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import tensorflow as tf
import pandas as pd
import numpy as np
import keras


%matplotlib inline

In [19]:
def Identity_block(input_tensor, kernel_size, filters, stage, block): 
    '''
    in here the F3 has to be the same as the number of the channels in input tensor 
    keep in mind that in this type nothing changes in the shape
    '''
    
    F1,F2,F3 = filters
    
    X = Conv2D(F1,(1,1),name="IC_1"+"_"+stage+"_"+block)(input_tensor)
    X = BatchNormalization(name="IB_1"+"_"+stage+"_"+block)(X)
    X = PReLU(name="IP_1"+"_"+stage+"_"+block)(X)
    
    X = Conv2D(F2,kernel_size=kernel_size,padding='same',name="IC_2"+"_"+stage+"_"+block)(X)
    X = BatchNormalization(axis=3,name="IB_2"+"_"+stage+"_"+block)(X)
    X = PReLU(name="IP_2"+"_"+stage+"_"+block)(X)
    
    X = Conv2D(F3,kernel_size=(1,1),name="IC_3"+"_"+stage+"_"+block)(X)
    X = BatchNormalization(axis=3,name="IB_3"+"_"+stage+"_"+block)(X)
    X = PReLU(name="IP_3"+"_"+stage+"_"+block)(X)
    
    X = Add(name="IA_4"+"_"+stage+"_"+block)([X,input_tensor])
    X = PReLU(name="IP_4"+"_"+stage+"_"+block)(X)
    
    return X


In [20]:
def Conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2,2)):
    F1, F2, F3 = filters
    
    
    X = Conv2D(F1,(1,1),strides=strides,name="CC_1"+"_"+stage+"_"+block)(input_tensor)
    X = BatchNormalization(axis=3,name="CB_1"+"_"+stage+"_"+block)(X)
    X = PReLU(name="CP_1"+"_"+stage+"_"+block)(X)
    
#     after this first layer it has already set shape to the width and the height
    
    
    X = Conv2D(F2,kernel_size=kernel_size,padding='same',name="CC_2"+"_"+stage+"_"+block)(X)
    X = BatchNormalization(axis=3,name="CB_2"+"_"+stage+"_"+block)(X)
    X = PReLU(name="CP_2"+"_"+stage+"_"+block)(X)
    
    
    X = Conv2D(F3,(1,1),name="CC_3"+"_"+stage+"_"+block)(X)
    X = BatchNormalization(axis=3,name="CB_3"+"_"+stage+"_"+block)(X)
    
#     after this third layer we have set tone to the number of channels as the ouput   
    
#     and the only reason we don't have to check the shape is that X1 - > X2 - > X3  and X_input - > X1 to begin with
    
#     because of the last two comments : strides = strides and F = F3 even for the shortcut of the input     
    
    SC = Conv2D(F3,(1,1),strides=strides,name="CSC_4"+"_"+stage+"_"+block)(input_tensor)
    SC = BatchNormalization(axis=3,name="CSP_4"+"_"+stage+"_"+block)(SC)
    
    
    X = Add(name="CSA_5"+"_"+stage+"_"+block)([X,SC])
    X = PReLU(name="CSP_5"+"_"+stage+"_"+block)(X)
    
    return X


In [1]:
def result(shape):
    X_input = Input(shape)
    X = X_input
    X = ZeroPadding2D((3,3))(X)
    X = Conv2D(64,(7,7),strides=(2,2),name="ConvDasti1")(X)
    X = BatchNormalization(axis=3,name='BatchDasti1')(X)
    X = PReLU(name="PreluDasti1")(X)
    X = MaxPooling2D((3,3),strides=(2,2),name="MaxPoolingDasti")(X)
    
    X = Conv_block(X,3,[64,64,256],stage='2',block='a',strides=(1,1))
    X = Identity_block(X,3,[64,64,256],stage='2',block='b')
    X = Identity_block(X,3,[64,64,256],stage='2',block='c')
    
    
    X = Conv_block(X,3,[128,128,512],stage='3',block='a')
    X = Identity_block(X,3,[128,128,512],stage='3',block='b')
    X = Identity_block(X,3,[128,128,512],stage='3',block='c')
    X = Identity_block(X,3,[128,128,512],stage='3',block='d')
    
    
    X = Conv_block(X,3,[256,256,1024],stage='4',block='a')
    X = Identity_block(X,3,[256,256,1024],stage='4',block='b')
    X = Identity_block(X,3,[256,256,1024],stage='4',block='c')
    X = Identity_block(X,3,[256,256,1024],stage='4',block='d')
    X = Identity_block(X,3,[256,256,1024],stage='4',block='e')
    X = Identity_block(X,3,[256,256,1024],stage='4',block='f')
    X = AveragePooling2D((2,2))(X)
    
    
    X = Flatten()(X)
    X = Dense(10,activation='softmax',name="final")(X)
    
    model = Model(inputs=X_input,outputs=X,name="model")
    
    return model


In [22]:
model = result([28,28,1])

In [23]:
model.compute_output_shape((42000,28,28,1))

(42000, 10)

In [24]:
df = pd.read_csv('train.csv')

In [25]:
label = df['label'].as_matrix()

In [26]:
label.shape

(42000,)

In [27]:
X = df.drop('label',axis=1).as_matrix()

In [28]:
X.shape

(42000, 784)

In [29]:
X1 = []
for i in range(X.shape[0]):
    X1.append(X[i,:].reshape(28,28,1))

In [32]:
resX = np.array(X1)

In [33]:
resX.shape

(42000, 28, 28, 1)

In [34]:
labels = []
with tf.Session() as sess:
    cl = tf.one_hot(label,10,axis=0)
    labels = sess.run(cl)

In [35]:
labels = labels.T

In [36]:
labels.shape

(42000, 10)

In [40]:
model.compile(optimizer=keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [41]:
model.fit(x=resX,y=labels,epochs=30,batch_size=32)

Epoch 1/30
42000/42000 [==============================] - 1766s - loss: 0.3870 - acc: 0.9118  
Epoch 2/30
42000/42000 [==============================] - 1615s - loss: 0.2058 - acc: 0.9514  
Epoch 3/30
42000/42000 [==============================] - 1616s - loss: 0.1295 - acc: 0.9691  
Epoch 4/30
42000/42000 [==============================] - 1618s - loss: 0.1067 - acc: 0.9725  
Epoch 5/30
42000/42000 [==============================] - 1618s - loss: 0.1718 - acc: 0.9608  
Epoch 6/30
42000/42000 [==============================] - 1617s - loss: 0.0744 - acc: 0.9800  
Epoch 7/30
42000/42000 [==============================] - 1614s - loss: 0.2063 - acc: 0.9640  
Epoch 8/30
42000/42000 [==============================] - 1618s - loss: 0.2720 - acc: 0.9547  
Epoch 9/30
42000/42000 [==============================] - 1613s - loss: 0.2218 - acc: 0.9560  
Epoch 10/30
42000/42000 [==============================] - 1609s - loss: 0.1802 - acc: 0.9659  
Epoch 11/30
42000/42000 [========================

In [42]:
model.save('Res50.h5')